# 大语言模型原理简介

**大语言模型**（**large language model**，**LLM**）的出现使得**人工智能**（**artificial intelligence**，**AI**）的发展进入了新的纪元，特别是OpenAI的GPT系列模型经过不断迭代进化，让人们看到了基于大语言模型的人工智能的无限可能。

实际上，大语言模型的出现并非突然，而是有深厚的“小语言”模型的基础。实际上正如**GPT**（**Generative Pre-trained Transformers**）包括**BERT**（**Bidirectional Encoder Representations from Transformers**）名称中所暗示的，两者都是基于谷歌的几位科学家2017年的论文《Attention is All You Need》所提出的Transformer架构：

![Transformer架构](pic/transformer.png)

而这一架构的提出则是将之前的**嵌入**（**embedding**）这一方法（如Word2Vec）进行了拓展。

在传统的嵌入方法中，句子首先被分解为一些**标记**（**token**）：
* 对于英文而言，标记可以是一个单词，或者是比单词更细的成分（比如把shocked分开为shock和ed）；
* 对于中文而言，通常每个字是一个token，或者可以通过分词技术进行分词（但在LLM中似乎没有必要）。
然后每个标记被转化为一个向量，从而把文本转化为了一个可以被计算的对象。

然而以上方法有其弊端：有的词天然是多义的，无法考虑上下文。比如，“学长可以写出更长的文章”这个句子中，两个“长”甚至连读音都不相同，但是在之前的嵌入模型中，“长”字可能被翻译为同一个向量。当然也许可以使用分词和n-gram的方法部分解决这一问题，然而会导致模型的复杂度急剧上升。

为此，Transformer架构中创造性地使用了**注意力**（**attention**）机制解决这一问题。我们在这里简要介绍一下注意力机制的简单直觉，不再对其数学部分做更多的解读，理解这一算法背后的直觉可以帮助我们理解大语言模型以及更好地使用大语言模型。

## 注意力机制

正如字面意思，注意力机制是模仿了人脑的注意力。比如，当我们阅读一段文字：

> 在我的后园，可以看见墙外有两株树，一株是枣树，还有一株也是枣树。

我们可以想象一下我们的大脑是如何处理这一句话的：当我们看到“两棵树”，自然会在后文中**查找**（**Query**）是哪两棵，我们可能会以“树”这个**关键字**（**Key**）去查找，然后找到了后文的两个“枣树”，进而在脑海中形成了“两颗枣树”这一映像，或者在计算机语言中，也就是“键”所对应的“**值**”（**Value**）。

在Attention中正是使用以上Q、K、V三个工具，通过Q和V两个矩阵相乘，经过Softmax变成一个注意力的概率，再与V相乘得到最终的注意力结果。

由于在一个句子中，根据不同的任务，我们的注意力可能集中在不同的token上面，所以在实际使用时，使用的是“多头注意力”（multi-head attention），即将多个注意力工具合在一起使用。注意力的架构如下图所示：

![Attention](pic/attention.png)

根据不同的任务，注意力可以分为两种：交叉注意力和自注意力。

* 交叉注意力（cross attention）：Q来自于一个序列，而K、V来自于另一个序列：

![Cross-Attention](pic/cross_attention.png)

* 自注意力（self-attention）：Q、K、V来自于同一个序列：

![Self-Attention](pic/self_attention.png)

## LLM的训练过程

注意力机制以及Transformer架构为大语言模型提供了技术保障，然而还有两个对LLM训练至关重要的问题没有解决：

1. 如何训练模型？需要定义好损失函数（训练目标是什么？）
2. 数据源从哪里来？

对第二个问题的回答来源于第一个问题，不同的训练任务、目标需要有不同的损失函数，从而也需要不同的数据。

在Transformer的原文中，主要解决的是翻译问题，那么只需要提供原文和译文，目标函数就是让机器根据原文翻译的译文与人翻译的尽量相近。为此，需要同时使用编码器和解码器。

而在Transformer提出后的一年，有两个工作放弃了编码器-解码器的结构，而是只使用了其中的一种：

* BERT：只使用了编码器
* GPT：只使用了解码器

具体而言，BERT由于只提供编码器作为预训练模型使用，因而只有预训练阶段；而GPT的目的是生成，因而除了预训练，还有有监督微调（supervised fine-tuning）。

在预训练阶段，BERT和GPT使用了类似的策略，其中：

* BERT完成两种任务
  1. 遮蔽语言模型（MLM），类似于完形填空，比如：
    > [CLS]一条大[MASK]波浪宽[SEP]
  2. 下句预测（NSP）：预测下个句子
* GPT主要预测下一个token（NTP）。

预训练之后，GPT还会进行进一步的有监督微调，通过文本分类、问答、自然语言推断等有监督任务继续训练模型。

BERT和GPT都不约而同使用了BooksCorpus的数据集，以及英文维基百科等。后续模型则扩充了使用的数据集。

从最初版本的GPT到GPT3，区别主要在于模型的大小和用于训练的数据量。

## 进化：从GPT到InstructGPT

经过以上的步骤之后，模型基本可以做到“熟读唐诗三百首，不会作诗也会吟”，然而其效果往往差强人意，离人工智能的目标还差的很远，甚至可能会产生对用户有害的输出。为此，OpenAI创造性地开创了InstructGPT。[这个链接](https://openai.com/index/instruction-following/) 解释了InstructGPT的工作流程，核心步骤如下：

![InstructGPT](pic/instructgpt.png)

其核心步骤包括两个：

1. 监督微调（supervised fine-tuning）
2. 通过人类反馈进行强化学习（reinforcement learning from human feedback, RLHF）

从第一个GPT模型只有0.1b参数（b代表billion，十亿）开始，到2020年GPT-3的175b参数，再到2022年InstructGPT推出，从而进化到GPT-3.5（同样为175b参数），大模型的逐渐展现出了超乎想象的能力。到现在的GPT-4、GPT-4o，短时间内大模型的推力能力得到了极大的增强。与此同时，各种各样的大模型不断推出，包括Llama、GLM、Bard以及中文的通义千问、文心一言等大模型也有不错的效果。

## 警惕AI幻觉

虽然大语言模型已经表现出了强大的推理能力，然而值得注意的是，这里的推理仍然是在语言层面而不是在逻辑层面。所谓推理，无非是根据上下文不断的对下一个token进行预测，借助于足够大的参数量，大语言模型往往可以“背过”很多答案，但是要求大语言模型可以基于逻辑进行推理现在仍然难以实现。当然，从另一个层面说，人类的思考也是借助于语言工具，也许有一天完全基于语言的大模型可以进行逻辑推理，但是现在仍然难以达到。

一个比较简单而又广为人知的例子是问大模型：9.9和9.11哪个大？包括GPT在内的很多模型都回答错误。如果使用一些提问的技巧，GPT甚至会“振振有词”地为错误答案辩护。在一些计算题中同样有这个问题，比如如果问2 * 2等于多少，可以得到正确答案，但是如果问3666 * 5444等于多少，则可能会给出错误答案。

因而，不要完全信任大语言模型给出的结果。在实际使用过程中，应该借助更多的工具帮助大语言模型得到更好的答案，而不是完全依赖这个答案。

## 提示工程

由于大语言模型的不完美，往往我们的问题得不到最满意的回答，然而通过提高提示词的质量往往可以得到更好的回答。为此，提供正确的、高质量的提示词就可以让我们对AI工具的使用事半功倍，这被称为提示工程（prompt engineering）。常见的方法有：

* 提供详细、明确的上下文
  > 有一家公司为大飞机生产起落架，是不是新质生产力？
  > 新质生产力是创新起主导作用，摆脱传统经济增长方式、生产力发展路径，具有高科技、高效能、高质量特征，符合新发展理念的先进生产力质态。它由技术革命性突破、生产要素创新性配置、产业深度转型升级而催生。以劳动者、劳动资料、劳动对象及其优化组合的跃升为基本内涵，以全要素生产率大幅提升为核心标志，特点是创新，关键在质优，本质是先进生产力。根据以上定义，请问为大飞机生产起落架的公司符不符合新质生产力的要求？
* 明确任务
  > 请为比亚迪公司是否属于新质生产力打分，1-10分。
* 明确角色
  > 如果你是一个计量经济学老师，请写一个教学计划。
* 零样本思维链策略（zero-shot-CoT strategy）：让LLM逐步思考、给出过程和步骤
  > 3666 * 5444等于多少？请给出计算步骤
* 少样本学习（few-shot learning）甚至单样本学习（one-shot learning）
  > 1025=1，20480=2，360560101=3，5023512004=？
* 格式化输出
  > 请为比亚迪公司是否属于新质生产力打分，1-10分，必须为数字
* 重复指示：提示词中多次加入相同的指令，但是表述方式不相同，也可以通过负面来提示
  > 你作为投资人，需要考察比亚迪公司的新质生产力发展情况，需要进行一个打分。请你根据你所知道的情况，对比亚迪公司进行分析，针对不好的地方进行扣分，1-10分。不要给出理由，必须给出数字。

## 思维链（Chain-of-Thought）

简单地使用LLM通常无法很好地完成算术、常识、符号等推理任务，为此Wei等（2022）提出了一个简单的方法可以极大程度提高推理效果，即**思维链**（**Chain of Thought**）

![InstructGPT](pic/CoT.png)

与以往的<input,output>的训练数据先比，CoT加入了<input,chain_of_thought,output>，通过过程引导就可以极大提高大模型的准确率。目前很多大语言模型特别是DeepSeek等都使用了思维链的方法。

除了思维链外，还有**思维树**（**Tree of Thought**），即产生多个推理分支，以树的形式组织推理过程。

如果有更加复杂的问题，在提示词中手动引导思维链、思维树也是不错的方法。

# Ollama的部署与使用

通常人们喜欢使用网页版大模型，然而很多时候这些大模型的使用缺乏定制，同时在任务量比较大时难以批量处理。为此，一个好的办法是直接使用大模型的接口。

比如，OpenAI提供了使用他们大模型的接口，在Python中可以直接使用
```bash
python3 -m pip install openai
```
进行安装和使用。当然，在使用前需要完成注册并获得秘钥。具体使用方法可以参考《大模型应用开发极简入门》一书。

OpenAI的工具有其优点，比如效果好，甚至可以定制化，但是也有其缺点，除了注册麻烦之外，使用OpenAI的成本也是非常高的。因而，在一些场景下，使用开源的大模型也许是一个不错的选择。

实际上开源大模型已经有很多，也达到了不错的效果。常见的有：

* `Llama`：Meta公司的大语言模型[Github](https://github.com/meta-llama/llama3) [Huggingface](https://huggingface.co/meta-llama)
* `gemma2`：Google的大语言模型[Huggingface](https://huggingface.co/google)
* `Phi-3`: 微软的大语言模型[Github](https://github.com/microsoft/Phi-3CookBook) [Huggingface](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf)
* `deepseek`: [Github](https://github.com/deepseek-ai/DeepSeek-R1) [Huggingface](https://huggingface.co/deepseek-ai/DeepSeek-R1)
* `通义千问`：阿里巴巴的大语言模型[Github](https://github.com/QwenLM) [Huggingface](https://huggingface.co/Qwen)
* `GLM`：智谱AI的大语言模型[Github](https://github.com/THUDM/GLM-4) [Huggingface](https://huggingface.co/THUDM)
* ......

此外还有一些多模态的模型（如llava）以及特定领域的模型（如代码生成和数学领域等）。

根据实际使用，开源大模型的完成度目前已经非常高，在特定领域甚至有一些优势，因而如果受限于条件和成本，特别是如果有微调大模型的需求，开源大模型也许是一个不错的选择。

## Ollama的部署

![Ollama](pic/ollama.png)

直接部署开源大模型往往比较麻烦，如果只是使用大模型而不是对大模型进行更深层次开发的话，使用一些成熟的部署工具就非常方便了。

这里以Ollama为例，介绍如何在本地部署大模型。

### 硬件环境

神经网络特别是大模型的训练和推理非常依赖显卡（GPU），而对于大模型而言，显存是重要的约束。比如，使用Ollama运行一个通义千问的14b模型，需要占用约11G显存，而往往很多人电脑的内存还到不了这个水平。

Ollama的一个好处是可以不依赖GPU就运行大模型，当然同样也需要足够大的内存。但是拥有一块具有足够大显存的显卡仍然是第一选择，因为GPU不管是训练还是推理，速度都远远快于CPU。

### 软件环境

* 操作系统：不限操作系统，但是推荐使用Linux
* Docker：一种称为“容器”的技术，类似于虚拟机但是有很大不同：使用Linux的命名空间（Namespaces）和控制组（Cgroups）完成容器和系统、容器之间的隔离，且不像虚拟机那样有大量性能消耗。Docker技术的使用极大方便了复杂环境的部署。
  * Windows和Mac也可以使用Docker，但是依靠的是虚拟一个Linux
  * Windows下也可以考虑通过WSL（Windows Subsystem for Linux）使用，但是WSL2同样基于虚拟机

### 安装

Ollama的[GitHub](https://github.com/ollama/ollama)上提供了下载链接：
* [Windows](https://ollama.com/download/OllamaSetup.exe)
* [Mac](https://ollama.com/download/Ollama-darwin.zip)
* Linux:
  ```bash
  curl -fsSL https://ollama.com/install.sh | sh
  ```

此外，Ollama还可以通过Docker安装：[镜像地址](https://hub.docker.com/r/ollama/ollama)，注意如果使用Docker安装需要区分CPU版和GPU版，GPU版需要安装NVIDIA的Container Toolkit。

### 拉取、运行模型

Ollama安装好之后，可以通过pull来拉取特定的模型，Ollama的官网提供了[模型列表](https://ollama.com/library)，比如拉取llama3.1：
```bash
sudo ollama pull llama3.1:8b
```
其中llama3.1是模型的名称，冒号后面的8b代表的是模型的大小。或者通义千问：
```bash
sudo ollama pull qwen3:14b
```

拉取好之后可以通过ollama run命令直接运行（不拉取直接run也会自动完成拉取步骤）：
```bash
ollama run qwen3:14b
```
然后就可以对话了。结束对话可以使用ctrl-D。

### 服务

对于Linux，安装好Ollama后会启动一个Ollama服务，该服务服务于**11434端口**，默认只侦听来自于本机(127.1.1.1或localhost)的访问。

如果需要设置Ollama能够接受所有的访问，需要对配置文件进行修改，比如在Linux中，需要
```bash
sudo vim /etc/systemd/system/ollama.service
```
在[Service]中加入：
```bash
Environment="OLLAMA_HOST=0.0.0.0"
```
然后
```bash
sudo systemctl daemon-reload
sudo systemctl restart ollama.service
```
即可。

Ollama的API使用方法可以在其[Github主页链接](https://github.com/ollama/ollama/blob/main/docs/api.md)上找到，我们接下来也会介绍主要接口的使用方法。

## 嵌入

大语言模型的一个基础就是嵌入。如前所述，借助于注意力机制，大语言模型可以通过上下文改变初始的token嵌入向量。传统上，BERT作为编码器是比较适合做嵌入工作的，不过基于解码器结构的生成式大语言模型也可以输出嵌入向量。

嵌入向量通常有如下作用：

* 在**检索增强生成**（**Retrieval-augmented Generation**）中作为检索依据
* 作为预训练模型继续参与接下来的训练（文本分类等问题）
* 在经济学中，结合双重机器学习，将文本作为控制变量时使用嵌入向量（如Dube，2020使用了Doc2Vec的嵌入结果）

借助于Ollama的接口，我们可以在不接触大模型源码的前提下直接得到嵌入向量的结果。

In [1]:
import urllib3
import json
import numpy as np


def request_ollama(data, api, url='127.0.0.1', port='11434'):
    """
    向Ollama发送请求
    :param data: 请求数据
    :param api: 请求的API
    :param url: 服务器地址
    :param port: 服务器端口
    :return: 服务器返回的数据
    """
    url = f'http://{url}:{port}/api/{api}'
    # 使用POST方法发送请求
    http = urllib3.PoolManager()
    response = http.request('POST',
                            url,
                            headers={'Content-Type': 'application/json'},
                            body=data)
    try:
        res = json.loads(response.data.decode('utf-8'))
    except Exception as e:
        res = response.data.decode('utf-8')
    return res


def get_ollama_embedding(text,
                         model='qwen2.5:14b',
                         url='127.0.0.1',
                         port='11434'):
    """
    获取ollama的embedding
    :param text: 输入文本
    :param model: 模型名称
    :param url: 服务器地址
    :param port: 服务器端口
    :return: embedding, 服务器返回的数据
    """
    data = json.dumps({"model": model, "input": text})
    res = request_ollama(data, 'embed', url, port)
    embedding = np.array(res['embeddings'])
    if embedding.ndim == 2:
        embedding = embedding.squeeze()
    return embedding, res


embedding, res = get_ollama_embedding(
    "大江东去，浪淘尽，千古风流人物。故垒西边，人道是，三国周郎赤壁。乱石穿空，惊涛拍岸，卷起千堆雪。江山如画，一时多少豪杰。遥想公瑾当年，小乔初嫁了，雄姿英发。羽扇纶巾，谈笑间，樯橹灰飞烟灭。故国神游，多情应笑我，早生华发。人生如梦，一尊还酹江月。"
)
print(embedding.shape)
print(embedding)

(5120,)
[-0.00446368  0.00065879  0.00081877 ...  0.00083635 -0.0085141
  0.01034792]


作为一个简单的应用，我们比较一下苏轼的两首词分别与李清照、毛主席的词的余弦相似度：

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


embedding1, _ = get_ollama_embedding(
    "明月几时有？把酒问青天。不知天上宫阙，今夕是何年。我欲乘风归去，又恐琼楼玉宇，高处不胜寒。起舞弄清影，何似在人间。转朱阁，低绮户，照无眠。不应有恨，何事长向别时圆？人有悲欢离合，月有阴晴圆缺，此事古难全。但愿人长久，千里共婵娟。"
)
embedding2, _ = get_ollama_embedding(
    "寻寻觅觅，冷冷清清，凄凄惨惨戚戚。乍暖还寒时候，最难将息。三杯两盏淡酒，怎敌他、晚来风急！雁过也，正伤心，却是旧时相识。满地黄花堆积，憔悴损，如今有谁堪摘？守着窗儿，独自怎生得黑！梧桐更兼细雨，到黄昏、点点滴滴。这次第，怎一个愁字了得！"
)
embedding3, _ = get_ollama_embedding(
    "北国风光，千里冰封，万里雪飘。望长城内外，惟余莽莽；大河上下，顿失滔滔。山舞银蛇，原驰蜡象，欲与天公试比高。须晴日，看红装素裹，分外妖娆。江山如此多娇，引无数英雄竞折腰。惜秦皇汉武，略输文采；唐宗宋祖，稍逊风骚。一代天骄，成吉思汗，只识弯弓射大雕。俱往矣，数风流人物，还看今朝。"
)

print("苏东坡v.s.苏东坡",cosine_similarity(embedding, embedding1))
print("苏东坡v.s.李清照",cosine_similarity(embedding, embedding2))
print("苏东坡v.s.毛主席",cosine_similarity(embedding, embedding3))
print("苏东坡v.s.李清照",cosine_similarity(embedding1, embedding2))
print("苏东坡v.s.毛主席",cosine_similarity(embedding1, embedding3))
print("李清照v.s.毛主席",cosine_similarity(embedding2, embedding3))

苏东坡v.s.苏东坡 0.9788975277391258
苏东坡v.s.李清照 0.9350070203571802
苏东坡v.s.毛主席 0.9602046010918898
苏东坡v.s.李清照 0.9359086463807932
苏东坡v.s.毛主席 0.9587357624662017
李清照v.s.毛主席 0.9406613332865817


可以看到，我们可以几乎无样本地得到一个很好的文本分析结果。

## 补全

嵌入只是小试牛刀，真正的重头戏是对话。在介绍对话之前，我们可以先从最简单的补全开始。

**补全**（**completion**）是大语言模型最基础的功能，即用户提出一个**提示**（**prompt**），由大语言模型回答。

对于Ollama，可以使用`/api/generate`接口完成，使用时需要使用POST方法以JSON的形式提供如下参数：

* `model`：所使用的模型，必须提供
* `prompt`：提示词
* `suffix`：设定模型响应结果的结尾
* `images`：对于多模态模型可以提供图片

此外，还有一些其他的选项可以使用，比如：

* `stream`：默认以流的方式响应，比较麻烦，可以直接设为false
* `system`：系统消息，用于设定模型行为
* `context`：上一次请求返回的context结果，可以用于维护一个简短的上下文
* `options`：一些影响模型行为的选项，包括：
  * `temperature`：温度，设定为0则同样的输入很有可能出现相同的结果（只使用预测概率最高的token），温度越高，结果随机性越强（越有创造性）
  * `top_p`：生成下一个token时，top_p指定一个累计概率阈值$p$，模型保留概率分布中累积概率之和达到$p$的最小子集作为候选集。top_p越小时，生成的文本越连贯、越符合预期，但是更缺乏创造性
  * `top_k`：在生成下一个单词时，top_k指定了从概率分布中保留概率最高的$k$个单词作为候选集。top_k越小，则生成的文本随机性越弱，生成的文本越连贯、越符合预期，但是更缺乏创造性
  * `seed`：随机数种子。设定之后每次生成结果相同。
* `keep_alive`：模型停留的时间，超过这个时间模型自动退出（默认`5m`）

比如，下面给出了一个简单的示例：

In [3]:
def get_ollama_completion(prompt,
                          suffix=None,
                          system=None,
                          temperature=0.5,
                          top_p=0.5,
                          seed=None,
                          context=None,
                          model='qwen2.5:14b',
                          url='127.0.0.1',
                          port=11434):
    """
    获取ollama的completion
    :param prompt: 输入文本
    :param suffix: 后缀文本 (可选)
    :param system: 系统信息 (可选)
    :param temperature: 温度默认为0.5 (可选)
    :param top_p: top_p 默认为0.5 (可选)
    :param seed: 随机种子 (可选)
    :param context: 上下文 (可选)
    :param model: 模型名称
    :param url: 服务器地址
    :param port: 服务器端口
    :return: 服务器返回的数据
    """
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "top_p": top_p
        }
    }
    if suffix is not None:
        data['suffix'] = suffix
    if seed is not None:
        data['options']['seed'] = seed
    if system is not None:
        data['system'] = system
    if context is not None:
        data['context'] = context
    data = json.dumps(data)
    res = request_ollama(data, 'generate', url, port)
    return res


response = get_ollama_completion(
    "如果一个数据库存储学生成绩，字段名name为姓名，id为学号，english为英语成绩，math为数学成绩，请写出查询语句，查询所有学生的姓名和数学成绩，结果按数学成绩降序排列。必须直接给出查询语句"
)
print(response)

{'model': 'qwen2.5:14b', 'created_at': '2024-11-05T11:44:15.269471703Z', 'response': '```sql\nSELECT name, math FROM student ORDER BY math DESC;\n```', 'done': True, 'done_reason': 'stop', 'context': [151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 62244, 46944, 74393, 105653, 47764, 43959, 99923, 3837, 44931, 13072, 606, 17714, 66187, 3837, 307, 17714, 47764, 17992, 3837, 29120, 17714, 104105, 100716, 3837, 10374, 17714, 104552, 100716, 37945, 112672, 51154, 72881, 99700, 3837, 51154, 55338, 105683, 66187, 33108, 104552, 100716, 3837, 59151, 59879, 104552, 100716, 99457, 32044, 108467, 1773, 100645, 101041, 107485, 51154, 72881, 99700, 151645, 198, 151644, 77091, 198, 73594, 3544, 198, 4858, 829, 11, 6888, 4295, 5458, 15520, 7710, 6888, 16089, 280, 73594], 'total_duration': 5279753597, 'load_duration': 4953578207, 'prompt_eval_count': 85, 'prompt_eval_duration': 54377000, 'eval_count': 16, 'ev

可以看到返回的Json数据中比较重要的信息有：

* `response`：返回的输出结果
* `done`：是否以完成（stream方式需要分多次返回结果，可能返回false
* `context`：上下文，可以用于下一次generate的`context`参数

其他的主要是一些计时、计数的信息。

`context`可以反复调用，比如写一首诗：

In [4]:
poetry = "思源湖\n"
response = get_ollama_completion(
    "你是上海对外经贸大学的学生，你热爱自己的母校，你想以你学校的思源湖为名字写一首五言诗，请写出第一句，必须直接写出诗句。",
    seed=1900,
    temperature=0.8,
    top_p=0.9)
poetry += response['response'] + '\n'

for i in range(3):
    response = get_ollama_completion(
        "你是上海对外经贸大学的学生，你热爱自己的母校，你想以你学校的思源湖为名字写一首五言诗，请继续写下一句，必须直接写出诗句。",
        temperature=0.7,
        top_p=0.6,
        context=response['context'])
    poetry += response['response'] + '\n'

print(poetry)

思源湖
思源湖水静，浟湙接天清。
晨风拂柳细，波映读书声。
鱼跃学海阔，雁点文澜轻。
碧波涵远志，桃李共春荣。



## 聊天

与补全相比，聊天需要额外的维护聊天的上下文，而这是通过我们“手动”记录每次聊天记录实现的。

为了区分聊天记录中每句话究竟是谁说的，我们就需要区分不同的“**角色**”（**role**）。在Ollama中，有几种不同的角色：

* `system`：系统，用于帮助设置助手的行为，或者放入一些背景信息
* `user`：用户，我们自己的提示词
* `assistant`：助手，大模型曾经的回复历史
* `tool`：工具，通过定义函数等丰富大模型的行为

聊天可以通过Ollama的`/api/chat`接口完成，其基本使用方法与generate几乎一样，唯一不同的是message不再是一条纯文本，而是一个列表，列表的每个元素都是一个包含有"role"和"content"两个key的字典。比如：

```json
"message": [
    {
        "role": "user",
        "content": "唐朝最著名的诗人是谁？请直接写出名字"
    },
    {
        "role": "assistant",
        "content": "李白"
    },
    {
        "role": "user",
        "content": "他最有名的诗是哪一首？请写出全文"
    }
]
```

如此就维护了一个聊天的上下文。

比如，我们可以构建如下对象进行聊天：

In [5]:
class OllamaChatAgent:
    """
    聊天代理
    """

    def __init__(self,
                 system='',
                 model='qwen2.5:14b',
                 url='127.0.0.1',
                 port='11434'):
        """
        初始化
        :param system: 系统信息 (可选)
        :param model: 模型名称
        :param url: 服务器地址
        :param port: 服务器端口
        """
        self.model = model
        self.url = url
        self.port = port
        self.message = [{"role": "system", "content": system}]

    def chat(self, prompt, temperature=0.5, top_p=0.5, seed=None):
        """
        获取ollama的chat
        :param prompt: 输入文本
        :param temperature: 温度默认为0.5 (可选)
        :param top_p: top_p 默认为0.5 (可选)
        :param seed: 随机种子 (可选)
        :return: 服务器返回的数据
        """
        self.message.append({"role": "user", "content": prompt})
        data = {
            "model": self.model,
            "messages": self.message,
            "stream": False,
            "options": {
                "temperature": temperature,
                "top_p": top_p
            }
        }
        if seed is not None:
            data['options']['seed'] = seed
        data = json.dumps(data)
        res = request_ollama(data, 'chat', self.url, self.port)
        # 将聊天记录添加到message中
        self.message.append(res['message'])
        return res['message']['content']


ocagent = OllamaChatAgent(
    system='L1范数即一个向量的所有元素绝对值之和，L2范数即一个向量的所有元素的平方和再开方，而L∞范数则是所有元素绝对值的最大值。')
print("第1次对话：")
print(ocagent.chat("向量[1,-1,3]的L1范数是多少？"))
print("-------------------------------------")
print("此时的message：")
print(ocagent.message)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("第2次对话：")
print(ocagent.chat("以上向量的L∞范数是多少？"))
print("-------------------------------------")
print("此时的message：")
print(ocagent.message)

第1次对话：
对于向量 \([1, -1, 3]\)，其 L1 范数是该向量中每个元素的绝对值之和。

计算如下：
\[ |1| + |-1| + |3| = 1 + 1 + 3 = 5 \]

因此，向量 \([1, -1, 3]\) 的 L1 范数为 \(5\)。
-------------------------------------
此时的message：
[{'role': 'system', 'content': 'L1范数即一个向量的所有元素绝对值之和，L2范数即一个向量的所有元素的平方和再开方，而L∞范数则是所有元素绝对值的最大值。'}, {'role': 'user', 'content': '向量[1,-1,3]的L1范数是多少？'}, {'role': 'assistant', 'content': '对于向量 \\([1, -1, 3]\\)，其 L1 范数是该向量中每个元素的绝对值之和。\n\n计算如下：\n\\[ |1| + |-1| + |3| = 1 + 1 + 3 = 5 \\]\n\n因此，向量 \\([1, -1, 3]\\) 的 L1 范数为 \\(5\\)。'}]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
第2次对话：
对于向量 \([1, -1, 3]\)，其 L∞ 范数是该向量中所有元素绝对值的最大值。

计算如下：
\[ |1| = 1,\quad |-1| = 1,\quad |3| = 3 \]

其中最大的绝对值为 \(3\)，因此向量 \([1, -1, 3]\) 的 L∞ 范数为 \(3\)。
-------------------------------------
此时的message：
[{'role': 'system', 'content': 'L1范数即一个向量的所有元素绝对值之和，L2范数即一个向量的所有元素的平方和再开方，而L∞范数则是所有元素绝对值的最大值。'}, {'role': 'user', 'content': '向量[1,-1,3]的L1范数是多少？'}, {'role': 'assistant', 'content': '对于向量 \\([1, -1, 3]\\)，其 L1 范数是该向量中每个元素的绝对值之

# 微调大模型

尽管大语言模型已经表现出了相当强大的能力，然而仍然有极大的优化空间。特别是对于特定领域的任务，对大模型进行**微调**（**fine-tuning**）可以进一步提高大模型的性能。

商业大模型一般可定制型较差，不过有的也提供了微调的工具。比如，OpenAI有专门针对基础模型进行微调的工具。OpenAI允许用户使用如下类型数据进行微调：
```json
{"prompt": "<prompt text>", "completion": "<completion text>"}
{"prompt": "<prompt text>", "completion": "<completion text>"}
{"prompt": "<prompt text>", "completion": "<completion text>"}
...
```

而开源大模型由于源代码和权重都是公开的，往往更容易在其基础上进行进一步微调。比如，Llama模型可以通过**指令微调**（**instruction tuning**）和**对话微调**（**conversation tuning**）两种方法来微调大模型。

## 指令微调

指令微调需要提供类似如下数据：
```json
{
    "instruction": "什么是累积分布函数？",
    "input": "",
    "output": "对于随机变量X，累积分布函数的定义为F\\(x\\)=P\\(X<=x\\)，即随机变量X小于等于x的概率。"
}
```
其中：
* `instruction`：指令，对任务的描述
* `input`：输入大模型的数据
* `output`：输出结果


## 对话微调

对话微调是一种特殊的指令微调，需要提供对话数据，比如：
```plaintext
<s>Human: 什么是内生性？</s><s>Assistant: 内生性是指解释变量与误差项相关，通常造成内生性的原因有：遗漏变量、反向因果、度量误差、自选择等。</s>
```

## 微调方法

一般微调可以使用全参数微调，也可以使用**LoRA**（**Low-rank Adaptation**）等方法进行高效微调。


对大模型的微调需要高质量的数据和更高要求的硬件，在这里不再进一步展开。

# Dify的部署和使用

以上基于Ollama的大语言模型开发应用流程虽然功能强大，但是仍然需要一些代码，在构建更加复杂、涉及多接口的任务时仍然略显繁琐。随着大语言模型应用需求增加，一些方便的大模型应用开发平台应用而生，如LangChain、Dify、Coze等。在这些应用平台中，往往已经封装好了很多应用程序接口，可以方便地直接调用。与此同时，通过工作流等机制可以在不适用代码的情况下就可以完成应用开发。

在这里，我们以Dify为例，介绍基于LLM的应用开发平台的使用方法。

## Dify的部署

Dify包含服务本身以及数据库、web服务等多个组件，因而从头部署非常复杂。幸运的是，借助Docker这一工具，可以非常方便省事地完成部署。

整个部署的步骤如下：

1. 安装Docker以及Docker compose
   ```bash
   sudo apt install docker.io docker-compose
   ```
   额外说明：Docker同样可以从远程拉取（pull）镜像，但是这个地址很难直接访问，如果有能scientifically上网工具作为代理的话，可以通过如下方法解决：
   * 编辑```/lib/systemd/system/docker.service```，在```[service]```中加入：
     ```sh
     [Service]
     Environment="HTTP_PROXY=http://192.168.1.16:8123"
     Environment="HTTPS_PROXY=http://192.168.1.16:8123"
     ```
     其中192.168.1.16为代理服务器地址，8123为代理的端口号。
2. 按照[官网指南](https://docs.dify.ai/zh-hans/getting-started/install-self-hosted/docker-compose)的指示进行安装
   1. 克隆Dify代码
      ```bash
      git clone https://github.com/langgenius/dify.git
      ```
    2. 复制配置文件（如果需要也可以修改）
      ```bash
      cd dify/docker
      cp .env.example .env
      ```
    3. 拉取、启动Docker容器
      ```bash
      sudo docker compose up -d
      ```

经过以上步骤之后，使用命令
```bash
sudo docker ps
```
可以查看运行状态，应该有9个容器都是Up的状态。

部署好Dify后，可以直接在浏览器输入```http://127.0.0.1/```访问主界面，第一次访问时需要设置一下管理员账户，然后就可以正常使用了。

## 模型

既然是基于LLM的应用，首先应该做的就是就是设定大语言模型。在Dify中，大多数商业的、开源的大模型都可以添加进去。

* 对于商业模型，通常需要设置API Key。如果不知道从哪里获得API Key，可以直接在Dify找到相关链接。
* 对于部署在本地的开源模型，只需要设定好地址、端口和模型即可。

设置方法：右上角用户名处，找到设置，模型供应商。

在Dify中，有四种模型可以被添加：

* 推理模型：也就是能够进行对话的模型
* Embedding模型：可以用于生成嵌入向量的模型
* Rerank模型：用于增强检索能力（后续介绍）
* 语言转文字模型：将语言转为文字

比如，如果我们在Ollama中加入了```qwen2.5```模型，自然就有了推理模型和Embedding模型：实际上我们在Ollama中介绍的```/api/chat```和```/api/embed```就分别对应了这里的两种模型。

## 应用类型

在Dify中提供了四种应用类型：

* 聊天助手：基于 LLM 构建对话式交互的助手
* 文本生成：构建面向文本生成类任务的助手，例如撰写故事、文本分类、翻译等
* 智能体（Agent）：能够分解任务、推理思考、调用工具的对话式智能助手
* 工作流：基于流程编排的方式定义更加灵活的LLM工作流

具体使用方法可以参考Dify的[文档](https://docs.dify.ai/zh-hans/guides/application-orchestrate)。

> 示例-政策不确定性

## 一个示例：基于LLM的情感分析

我们使用大众点评的评论数据，使用LLM对评论进行了情感分析，详见```大众点评情感分析```。由于数据集太大，我们只使用了50条评论（```csv/dianping_keep_comment.csv```）。可以方便地使用Dify的批量处理得到结果。最终结果可以看```Dify/大众点评情感分析结果.csv```。

## 工具

在Dify中内嵌了非常多方便使用的工具，这些工具有些需要授权，有些不需要授权，甚至可以自己定义一些工具来使用，非常方便。比如在以上政策不确定性的例子中，我们就使用了```网页抓取```这个小工具。

当然，有的时候这些工具不足以完成我们的工作，此时也许需要我们自己手写代码。同样Dify可以支持自己手写Python或者JavaScript代码。如果需要额外安装包的话，需要使用：
```sh
sudo docker ps
```
找到```langgenius/dify-sandbox:*****```前面对应的CONTAINER ID，然后使用命令：
```sh
sudo docker exec -it {CONTAINER ID} python3 -m pip install bs4
```
其中```{CONTAINER ID}```为刚刚找到的CONTAINER ID，```bs4```为拟安装的包。

> 示例-上海统计表格整理（https://tjj.sh.gov.cn/ydsj2/20240417/dba9c85c00c64c6bbdea657c69555625.html）

## 知识库

前面提到大语言模型无非是基于已学习样本的一种文本生成，无法认知没有学习的知识，当然也没有办法做完全的逻辑推理。微调可以部分解决这一问题，但是训练难度高，样本准备繁琐。那么能不能让大语言模型实时根据提示取寻找有用的信息呢？

**检索增强生成**（**Retrieval Augmented Generation**）提供了这样的一种技术路线。在该技术中，无需对大模型进行微调，而是可以根据提示词构建查询，找到与当前提示词最相关的知识，将找到的知识作为背景提供给大模型，以帮助大模型生成更好的结果。

所以这里面就涉及到如何检索的问题。Dify里面提供了以下检索方式：

* 向量检索。通过Embedding出的向量进行检索，所以需要Embedding模型
* 全文检索。通过关键词检索
* 混合检索。同时使用如上两种方法

给定检索出的很多结果，如何找到最有用的呢？这就需要用到**重排**（**Rerank**）模型了。由于一般的LLM不带重排模型，我们必须额外指定重排模型。有一些商用的重排模型，或者也可以商用开源免费的，比如Huggingface的```text-embeddings-inference```模型，[项目地址](https://github.com/huggingface/text-embeddings-inference?tab=readme-ov-file)。

安装过程：
```bash
# 设置代理
# export NO_PROXY="127.0.0.1,localhost"
# export HTTP_PROXY=
# export HTTPS_PROXY=
# sudo apt install git-lfs # or 'git lfs install' on some systems 
git clone https://huggingface.co/BAAI/bge-reranker-large
sudo docker run -d -p 7777:80 --restart=always -v /home/aragorn/text-embeddings-inference/data:/data --pull always ghcr.io/huggingface/text-embeddings-inference:cpu-1.5 --model-id /data/bge-reranker-large
```
其中：
* export的三句是为了给huggingface.co设置代理（能正常访问就忽略）
* ```sudo apt install git-lfs```用于为git安装大文件支持
* ```git clone```用于克隆仓库
* 最后的docker命令用于启动镜像，其中
  * -d 为守护模式
  * -p 为端口映射，将镜像的80端口映射给系统的7777（随意指定）端口
  * -v 用于指定数据存放位置，将镜像内的/data映射给冒号前面的路径
  * --pull 拉取CPU版的镜像
  * --model-id 模型具体版本
  
以上步骤设置好之后，就可以在```知识库```中创建知识库，将所有的知识语料放在其中，设定好分段、索引方式、Embedding模型、检索方式后，Dify会自动构建知识库。

有了知识库之后，在实际使用中，知识库可以作为一个工具，通过查询知识库，把知识库的结果作为大语言模型的上下文使用。

> 党建机器人

# 智能体

基于大语言模型以及以上介绍的工具、知识库，可以进一步开发能够自主理解、做出决策、执行任务的的**智能体**（**Agent**）。智能体可以用下面的公式来概括：$$Agent=大语言模型+记忆+主动规划+工具使用$$

智能体的开发通常可以划分为3层：

* 基础层，即通用大语言模型，为智能体提供“智能”的部分
* 垂直/行业层，专注于特定领域或者问题，将基础层的能力进一步专业化、精细化
* 应用层，在以上两层的基础上提供服务

智能体开发：Coze和Dify中提供了很多案例供学习。

> 示例：[《统计学A》AI助教](https://www.coze.cn/store/agent/7521662804978958387?bot_id=true)